## Import inicial

In [0]:
import pyspark.sql.functions as F
import pyspark.pandas as ps
import pandas as pd
import numpy as np

In [0]:
df = spark.read.csv(
    "/Volumes/telecom/bronze/teleco_raw/WA_Fn-UseC_-Telco-Customer-Churn.xls",
    header=True,
    inferSchema=True,
)
df = df.toPandas()

In [0]:
df.head(10)

In [0]:
df.columns

## Data manipulation

Aqui podemos observar que a coluan  `TotalCharges` está sendo tratada como um objeto e não como um tipo númerico float

In [0]:
df.info()

É confirmado pelo `df.describe()` pois não conseguimos extrair estatisticas numéricas desta coluna.

In [0]:
df.describe().T

Então precisamos converter a coluna `TotalCharges` para um tipo numérico e verificar as linhas que apresentarem null/na

In [0]:
df['TotalCharges'] = pd.to_numeric(df.TotalCharges, errors='coerce')
df.isnull().sum() # 11 linhas NAN

Quais são as linhas que encontramos NA?

In [0]:
df[np.isnan(df['TotalCharges'])]

Observamos que enquanto `tenure` -  _meses que o cliente permaneceu na companhia_ - for **0**, o `TotalCharges` - _total de gasto do cliente_ - será **NaN**

In [0]:
df.groupby('tenure')[['TotalCharges']].agg(['mean', 'count'])

Para solucionar esses problemas,
- 1 - vamos eliminar as linhas de `tenure` = 0. Pois queremos apenas clientes que fidelizaram com a empresa por algum perído.
- 2 - Preencher `TotalCharges` com a média da coluna

In [0]:
linhas_nan = df[df['tenure'] == 0].index
print(f'quantidade de linhas NaN em tenure: {len(linhas_nan)}')


In [0]:
# removendo linhas com tenure == 0
df.drop(labels = linhas_nan, axis = 0, inplace = True)
df.isnull().sum()

In [0]:
df.fillna(df["TotalCharges"].mean())

In [0]:
df.groupby('tenure')[['TotalCharges']].agg(['mean', 'count'])

In [0]:
df.info()

## Salvando o trabalho

In [0]:
df_spark = spark.createDataFrame(df)
df_spark.write.format("delta").mode("overwrite").save("/Volumes/telecom/silver/teleco_cleaned")



In [0]:
df_spark.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("telecom.silver.teleco_cleaned")


In [0]:
%sql

SELECT * FROM telecom.silver.teleco_cleaned
LIMIT (30);